In [6]:

# Your program should print out the result of
# your two sense measurements.
#
# Don't modify the code below. Please enter
# your code at the bottom.

from math import *
import random



landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0


class robot:
    def __init__(self):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0;
        self.turn_noise    = 0.0;
        self.sense_noise   = 0.0;
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
            raise (ValueError, 'X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
            raise (ValueError, 'Y coordinate out of bound')
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise (ValueError, 'Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    
    def sense(self):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    
    def move(self, turn, forward):
        if forward < 0:
            raise (ValueError, 'Robot cant move backwards')         
        
        # turn, and add randomness to the turning command
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size    # cyclic truncate
        y %= world_size
        
        # set particle
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res
    
    def Gaussian(self, mu, sigma, x):
        
        # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))
    
    
    def measurement_prob(self, measurement):
        
        # calculates how likely a measurement should be
        
        prob = 1.0;
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob
    
    
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))



def eval(r, p):
    sum = 0.0;
    for i in range(len(p)): # calculate mean error
        dx = (p[i].x - r.x + (world_size/2.0)) % world_size - (world_size/2.0)
        dy = (p[i].y - r.y + (world_size/2.0)) % world_size - (world_size/2.0)
        err = sqrt(dx * dx + dy * dy)
        sum += err
    return sum / float(len(p))



####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####






In [7]:
# forwart_noise = 5.0, turn_noise = 0.1, sense_noise = 5.0
# starts at 30, 50 heading north (pi/2).
# Have your robot turn clockwise by pi/2, move
# 15 m, and sense. Then have it turn clockwise
# by pi/2 again, move 10 m, and sense again.
myrobot = robot()
myrobot.set_noise(5.0, 0.1, 5.0)
myrobot.set(30.0, 50.0, pi/2)
print(myrobot)
myrobot = myrobot.move(-pi/2, 15.0)
print(myrobot.sense())
myrobot = myrobot.move(-pi/2, 10.0)
print(myrobot.sense())


[x=30.0 y=50.0 orient=1.5707]
[36.99521515020645, 42.37975999156781, 38.79534364092275, 52.23881830505362]
[18.064326466676846, 60.57520655270855, 37.36115904707321, 49.12116866611251]


In [8]:
##create particles

myrobot = robot()


number_of_particles = 1000
particles = []

for i in range(number_of_particles):
    x = robot()
    x.set_noise(0.05,0.05, 5.0)
    particles.append(x)
T = 10
for t in range(T):
    myrobot = myrobot.move(0.1, 5.0)
    measurement = myrobot.sense()

    particles2 = []
    for particle in particles:
        particles2.append(particle.move(0.1, 5.0))

    particles = particles2

    weights = []

    for particle in particles:
        weights.append(particle.measurement_prob(measurement))


    res_particles = []
    index = int(random.random() * number_of_particles)
    beta = 0.0
    max_weight = max(weights)
    for i in range(number_of_particles):
        beta += random.random() * max_weight * 2
        while beta > weights[index]:
            beta -= weights[index]
            index = (index + 1) % number_of_particles
        res_particles.append(particles[index])
    
    

    particles = res_particles
    print(eval(myrobot, particles))


4.167228741816179
5.317010458337
5.400952895038833
4.665486546037629
3.322037414089273
2.291164237973452
1.9754297082914543
2.410205208818927
3.0975501159199363
2.3574512804179846


6.202124771932749
6.110584784132933
7.909843425697376
7.762304603086726
7.040963220429617
6.450637496653022
6.553702758133151
6.748922911566658
12.238360663246885
16.503267535338445


In [19]:
myrobot = robot()
move = [0.1, 5.0]
myrobot.move(move[0], move[1])

particles= []
number_of_particles = 1000
for i in range(number_of_particles):
    x = robot()
    x.set_noise(0.05, 0.05, 5.0)
    particles.append(x)
    
T =10
    
for t in range(T):
    myrobot.move(move[0], move[1])
    measurement = myrobot.sense()
    particles2 = []
    for particle in particles:
        particles2.append(particle.move(move[0], move[1]))
    
    particles = particles2
    weights = []
    for i in range(number_of_particles):
        weights.append(particles[i].measurement_prob(measurement))
    
    beta = 0.0
    max_weight =max(weights)
    resampling = []
    index = int(random.random() * number_of_particles)
    for i in range(number_of_particles):
        beta += random.random() * 2 * max_weight
        while(beta > weights[index]):
            beta -= weights[index]
            index = (index + 1) % number_of_particles
        resampling.append(particles[index])
    
    particles = resampling
    print(particles)
        

[[x=65.317 y=65.227 orient=3.0382], [x=65.317 y=65.227 orient=3.0382], [x=65.317 y=65.227 orient=3.0382], [x=65.317 y=65.227 orient=3.0382], [x=60.984 y=70.685 orient=0.2509], [x=71.528 y=63.290 orient=5.1640], [x=71.528 y=63.290 orient=5.1640], [x=68.318 y=61.730 orient=3.8002], [x=57.195 y=73.653 orient=5.7159], [x=74.512 y=65.211 orient=4.8113], [x=63.067 y=73.131 orient=2.2891], [x=65.317 y=65.227 orient=3.0382], [x=65.317 y=65.227 orient=3.0382], [x=68.014 y=78.127 orient=2.0428], [x=68.318 y=61.730 orient=3.8002], [x=74.512 y=65.211 orient=4.8113], [x=66.603 y=63.569 orient=3.3125], [x=66.603 y=63.569 orient=3.3125], [x=70.596 y=66.179 orient=2.4079], [x=65.317 y=65.227 orient=3.0382], [x=65.317 y=65.227 orient=3.0382], [x=60.338 y=63.742 orient=0.2860], [x=68.318 y=61.730 orient=3.8002], [x=63.067 y=73.131 orient=2.2891], [x=70.596 y=66.179 orient=2.4079], [x=65.317 y=65.227 orient=3.0382], [x=77.513 y=56.323 orient=4.6836], [x=81.553 y=64.486 orient=0.2106], [x=71.528 y=63.290 